In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%writefile .env

# TAVILY_API_KEY = your_key
# JINA_API_KEY = your_key
# CHROME_INSTANCE_PATH = /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
# BROWSER_HEADLESS=False

# Planning 부터 시작

In [ ]:
import sys
from src.workflow import run_agent_workflow

In [ ]:
# Choose workflow type: 'regular' or 'scm'
workflow_type = 'scm'  # Change to 'regular' for normal workflow

# SCM queries - Supply Chain Management examples
scm_user_query = '''
Analyze the impact of Chicago port strikes on our company's supply chain. 
Focus on lead time increases, transportation cost impacts, and potential effects on order fulfillment rates.
How should we adjust our logistics strategy and what alternative routes should we consider?
'''

scm_user_query = '''
시카고 항만 파업이 우리 회사 공급망에 미치는 영향을 분석해 주세요.
리드타임, 운송비 주문 충족 및 이행률, 재고일수, 현금화 주기에 미칠 잠재적 영향에 집중해 주세요.
우리는 물류 전략을 어떻게 조정해야 하며, 어떤 대체 경로를 고려해야 할까요?
./data 디렉토리에 나의 회사 관련 데이터가 있어. 그중 data_descriptions.txt는 각 데이터에 대한 설명이야. 
scm_researcher_node, scm_insight_analyzer_node, scm_impact_analyzer 이건 매우매우 간단히 해줘. scm_correlation_analyzer 테스트 중이거든
'''

# Regular analysis queries
regular_user_query = '''
AWS Cloud Market Analysis: Growth Prospects in Korea (2025-2030) 조사해줘. 결과는 pdf로 만들어줘, 
research와 coder는 정말 간단하게 1번씩만 호출 하고 조사도 아주 간단하게만 해. reporter 에이전트 테스트 중이거든
'''

# Select query based on workflow type
user_query = scm_user_query if workflow_type == 'scm' else regular_user_query

print(f"Selected workflow: {workflow_type}")
print(f"Query: {user_query[:100]}...")

# Other example queries (commented out)
'''
Other SCM example queries:
- "Analyze the impact of Suez Canal blockage on global semiconductor supply chain"
- "Assess the effects of Chinese port lockdowns on automotive industry supply chains"
- "Evaluate alternative shipping routes due to Red Sea disruptions"
- "Supply chain risk assessment for electronics manufacturing due to Taiwan tensions"

Other regular analysis queries:
- "LG 스타일러와 삼성 제품간의 비교가 필요해. 결과는 pdf로 만들어줘"
- "Transformer 알고리즘과 현 시점에서 어떤식으로 변화 발전 하는지 알려줘"
- "5-7세 아동을 위한 미술학원(홈스쿨)을 운영예정이야. 나는 미술치료와 심리학을 전공했어. 이런 전문성을 접목한 브랜딩, 마케팅 방법, 운영방안, 구체적인 프로그램(월별)을 기획해줘. 그리고 브랜드 이름도 추천해줘."
'''

In [ ]:
import os
import shutil
import nest_asyncio

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

nest_asyncio.apply()
remove_artifact_folder()

# Execute based on workflow type
if workflow_type == 'scm':
    print("🔗 Using SCM specialized workflow")
    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )
    
    # Print SCM workflow history
    print("\n=== SCM Analysis History ===")
    print("result", result)
    for history in result.get("history", []):
        print("===")
        print(f'agent: {history["agent"]}')
        print(f'message: {history["message"]}')
        
    # Show artifacts summary for SCM
    print(f"\n📁 Check ./artifacts/ folder for generated analysis files")
    if os.path.exists("./artifacts/"):
        files = [f for f in os.listdir("./artifacts/") if f.endswith('.txt')]
        print(f"Generated {len(files)} analysis files:")
        for file in sorted(files):
            print(f"  - {file}")

else:
    print("📊 Using regular analysis workflow")
    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )

    # Print the conversation history
    print("\n=== Conversation History ===")
    print("result", result)
    for history in result["history"]:
        print("===")
        print(f'agent: {history["agent"]}')
        print(f'message: {history["message"]}')

# SCM (Supply Chain Management) Analysis

이 섹션은 공급망 분석 전용 워크플로입니다. 

## SCM 워크플로 특징:
- **전문 에이전트**: SCM 전문가 에이전트들이 단계별 분석 수행
- **파일 기반**: 각 단계별 분석 결과를 `./artifacts/` 폴더에 저장
- **연구 출처**: 모든 연구 결과에 URL 출처 포함
- **KPI 분석**: 공급망 KPI 정량 분석 및 상관관계 분석
- **대응 전략**: 구체적인 공급망 리스크 대응 방안 제시

## 주요 분석 영역:
- 항만 파업, 운하 봉쇄 등 물류 중단 이벤트
- 운송비 증가, 리드타임 연장 등 비용 영향
- 공급업체 다변화, 대체 루트 등 대응 전략

In [ ]:
# SCM Quick Test - 간단한 SCM 분석 테스트
import os
import shutil
import nest_asyncio
from src.workflow import run_scm_workflow

# 빠른 SCM 테스트 쿼리들
scm_queries = {
    "chicago_port": "Analyze the impact of Chicago port strikes on our supply chain",
    "suez_canal": "Assess the supply chain impact of Suez Canal disruptions on electronics industry", 
    "china_lockdown": "Evaluate the effects of Chinese port lockdowns on automotive supply chains",
    "red_sea": "Analyze alternative shipping routes due to Red Sea security concerns"
}

# 테스트할 쿼리 선택
selected_query = "chicago_port"  # 원하는 쿼리 키로 변경
user_query = scm_queries[selected_query]

print(f"🔗 Testing SCM query: {selected_query}")
print(f"📝 Query: {user_query}")

# 아티팩트 폴더 정리
def remove_artifact_folder(folder_path="./artifacts/"):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Cleared {folder_path}")

nest_asyncio.apply()
remove_artifact_folder()

# SCM 워크플로 실행
result = run_scm_workflow(
    user_input=user_query,
    debug=False
)

print("\n✅ SCM Analysis Complete!")
print(f"📁 Check ./artifacts/ folder for generated files")